# 🔮 CAPTCHA OCR Prediction - Kaggle Version

Этот notebook предназначен для запуска предсказаний на Kaggle с использованием обученной модели.

## Что нужно сделать перед запуском:
1. Загрузить обученную модель в `input/` (файл `model.keras`)
2. Загрузить тестовые данные в `input/` (папка с изображениями)
3. Запустить все ячейки

## Структура input:
```
input/
├── model.keras          # Обученная модель
└── test_images/         # Папка с тестовыми CAPTCHA
    ├── captcha_001.png
    ├── captcha_002.png
    └── ...
```

In [ ]:
# Импорт необходимых библиотек
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from PIL import Image
import glob
import pandas as pd
from IPython.display import display, HTML

print("📦 Библиотеки загружены успешно")
print(f"TensorFlow версия: {tf.__version__}")
print(f"Keras версия: {keras.__version__}")

## 🔧 Настройки и конфигурация

In [ ]:
# Настройки модели
IMG_WIDTH = 200
IMG_HEIGHT = 60
MAX_SEQUENCE_LENGTH = 7

# Пути к данным
MODEL_PATH = "/kaggle/input/model.keras"
TEST_IMAGES_PATH = "/kaggle/input/test_images/"
OUTPUT_PATH = "/kaggle/working/predictions/"

# Создаем папку для результатов
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("⚙️ Настройки загружены")
print(f"Путь к модели: {MODEL_PATH}")
print(f"Путь к тестовым изображениям: {TEST_IMAGES_PATH}")
print(f"Папка для результатов: {OUTPUT_PATH}")

## 🔍 Проверка доступных файлов

In [ ]:
# Проверяем доступные файлы
print("📁 Содержимое input:")
for root, dirs, files in os.walk("/kaggle/input/"):
    level = root.replace("/kaggle/input/", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in files[:10]:  # Показываем только первые 10 файлов
        print(f"{subindent}{file}")
    if len(files) > 10:
        print(f"{subindent}... и еще {len(files) - 10} файлов")

# Проверяем наличие модели
if os.path.exists(MODEL_PATH):
    print(f"\n✅ Модель найдена: {MODEL_PATH}")
    model_size = os.path.getsize(MODEL_PATH) / (1024 * 1024)
    print(f"   Размер: {model_size:.2f} MB")
else:
    print(f"\n❌ Модель не найдена: {MODEL_PATH}")
    print("   Убедитесь, что загрузили файл model.keras в input/")

# Проверяем наличие тестовых изображений
if os.path.exists(TEST_IMAGES_PATH):
    image_files = glob.glob(os.path.join(TEST_IMAGES_PATH, "*.png")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpg")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpeg"))
    print(f"\n✅ Тестовые изображения найдены: {len(image_files)} файлов")
    if image_files:
        print(f"   Примеры: {[os.path.basename(f) for f in image_files[:5]]}")
else:
    print(f"\n❌ Папка с тестовыми изображениями не найдена: {TEST_IMAGES_PATH}")
    print("   Убедитесь, что загрузили папку с изображениями в input/test_images/")

## 🧠 Загрузка модели и словарей

In [ ]:
def load_model(model_path):
    """Загружает обученную модель"""
    try:
        print(f"🔄 Загружаем модель из {model_path}...")
        model = keras.models.load_model(model_path)
        print("✅ Модель успешно загружена")
        return model
    except Exception as e:
        print(f"❌ Ошибка загрузки модели: {e}")
        return None

def load_char_mappings():
    """Загружает словари символов"""
    # Пытаемся определить алфавит из данных обучения
    characters = None
    
    # Проверяем, есть ли файл с метками в input
    possible_labels = [
        "/kaggle/input/test_images/labels.csv",
        "/kaggle/input/labels.csv", 
        "data/labels.csv",
        "labels.csv"
    ]
    
    for labels_file in possible_labels:
        if os.path.exists(labels_file):
            try:
                print(f"🔍 Определяем алфавит из {labels_file}...")
                with open(labels_file, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                
                # Извлекаем все символы из данных
                all_chars = set()
                for line in lines:
                    text = line.split(';')[0].strip()
                    all_chars.update(text)
                
                characters = sorted(all_chars)
                print(f"✅ Алфавит определен из данных: {len(characters)} символов")
                break
            except Exception as e:
                print(f"⚠️ Ошибка чтения {labels_file}: {e}")
                continue
    
    # Если не удалось определить из данных, используем полный алфавит
    if characters is None:
        print("⚠️ Не удалось определить алфавит из данных, используем полный набор")
        characters = sorted(set('абвгдежзийклмнопрстуфхцчшщъыьэюя0123456789'))
    
    char_to_num = {v: i for i, v in enumerate(characters)}
    num_to_char = {str(i): v for i, v in enumerate(characters)}
    num_to_char['-1'] = 'UKN'  # Для CTC декодирования
    
    print(f"📝 Алфавит ({len(characters)} символов): {characters}")
    return char_to_num, num_to_char

# Загружаем модель
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    char_to_num, num_to_char = load_char_mappings()
    
    # Показываем архитектуру модели
    if model:
        print("\n🏗️ Архитектура модели:")
        model.summary()
else:
    print("❌ Не удалось загрузить модель - файл не найден")
    model = None

## 🖼️ Функции предобработки изображений

In [ ]:
def preprocess_image(image_path, img_width=IMG_WIDTH, img_height=IMG_HEIGHT):
    """Предобрабатывает изображение для модели"""
    try:
        # Загружаем изображение
        img = tf.io.read_file(image_path)
        
        # Декодируем (пробуем PNG, затем JPEG)
        try:
            img = tf.io.decode_png(img, channels=3)
        except:
            img = tf.io.decode_jpeg(img, channels=3)
        
        # Конвертируем в float32 и нормализуем
        img = tf.image.convert_image_dtype(img, tf.float32)
        
        # Транспонируем (width, height, channels)
        img = tf.transpose(img, perm=[1, 0, 2])
        
        # Изменяем размер если нужно
        img = tf.image.resize(img, [img_width, img_height])
        
        # Добавляем batch dimension
        img = tf.expand_dims(img, 0)
        
        return img.numpy()
    except Exception as e:
        print(f"❌ Ошибка предобработки изображения {image_path}: {e}")
        return None

def decode_predictions(predictions, num_to_char):
    """Декодирует предсказания модели в текст"""
    # Получаем индексы с максимальной вероятностью
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    results = tf.keras.backend.ctc_decode(predictions, input_length=input_len, greedy=True)[0][0]
    
    # Конвертируем в текст
    texts = []
    for result in results:
        text = ""
        for idx in result:
            if idx != -1:  # -1 означает padding
                char = num_to_char.get(str(idx.numpy()), '')
                if char != 'UKN':
                    text += char
        texts.append(text)
    
    return texts

print("🔧 Функции предобработки загружены")

## 🔮 Функции предсказания

In [ ]:
def predict_single_image(model, image_path, char_to_num, num_to_char):
    """Предсказывает текст для одного изображения"""
    filename = os.path.basename(image_path)
    print(f"🔍 Анализируем: {filename}")
    
    # Предобрабатываем изображение
    processed_img = preprocess_image(image_path)
    if processed_img is None:
        return None
    
    # Делаем предсказание
    try:
        predictions = model.predict(processed_img, verbose=0)
        predicted_text = decode_predictions(predictions, num_to_char)[0]
        
        print(f"📝 Предсказанный текст: '{predicted_text}'")
        return predicted_text
        
    except Exception as e:
        print(f"❌ Ошибка предсказания: {e}")
        return None

def visualize_prediction(image_path, predicted_text, save_path=None):
    """Визуализирует изображение с предсказанием"""
    try:
        # Загружаем изображение для отображения
        img = Image.open(image_path)
        
        # Создаем фигуру
        plt.figure(figsize=(10, 4))
        plt.imshow(img, cmap='gray')
        plt.title(f"Предсказание: '{predicted_text}'", fontsize=16, fontweight='bold')
        plt.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            print(f"💾 Результат сохранен в {save_path}")
        
        plt.show()
        
    except Exception as e:
        print(f"❌ Ошибка визуализации: {e}")

print("🔮 Функции предсказания загружены")

## 🧪 Тестирование на одном изображении

In [ ]:
# Находим первое доступное изображение для тестирования
if os.path.exists(TEST_IMAGES_PATH) and model:
    image_files = glob.glob(os.path.join(TEST_IMAGES_PATH, "*.png")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpg")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpeg"))
    
    if image_files:
        test_image = image_files[0]
        print(f"🧪 Тестируем на изображении: {os.path.basename(test_image)}")
        
        # Делаем предсказание
        predicted_text = predict_single_image(model, test_image, char_to_num, num_to_char)
        
        if predicted_text is not None:
            # Показываем результат
            visualize_prediction(test_image, predicted_text)
        else:
            print("❌ Не удалось сделать предсказание")
    else:
        print("❌ Изображения не найдены в папке test_images")
else:
    print("❌ Модель или папка с изображениями не найдены")

## 📊 Массовое предсказание

In [ ]:
if os.path.exists(TEST_IMAGES_PATH) and model:
    # Находим все изображения
    image_files = glob.glob(os.path.join(TEST_IMAGES_PATH, "*.png")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpg")) + \
                  glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpeg"))
    
    if image_files:
        print(f"📊 Обрабатываем {len(image_files)} изображений...")
        
        # Обрабатываем каждое изображение
        results = []
        successful_predictions = 0
        
        for i, image_path in enumerate(image_files):
            predicted_text = predict_single_image(model, image_path, char_to_num, num_to_char)
            
            if predicted_text is not None:
                results.append({
                    'filename': os.path.basename(image_path),
                    'prediction': predicted_text,
                    'length': len(predicted_text)
                })
                successful_predictions += 1
            else:
                results.append({
                    'filename': os.path.basename(image_path),
                    'prediction': '',
                    'length': 0
                })
            
            # Показываем прогресс
            if (i + 1) % 10 == 0:
                print(f"   Обработано: {i + 1}/{len(image_files)}")
        
        print(f"\n✅ Обработка завершена!")
        print(f"   Успешных предсказаний: {successful_predictions}/{len(image_files)}")
        print(f"   Процент успеха: {successful_predictions/len(image_files)*100:.1f}%")
        
        # Создаем DataFrame с результатами
        df_results = pd.DataFrame(results)
        
        # Показываем статистику
        print("\n📈 Статистика предсказаний:")
        print(f"   Средняя длина предсказания: {df_results['length'].mean():.1f} символов")
        print(f"   Минимальная длина: {df_results['length'].min()} символов")
        print(f"   Максимальная длина: {df_results['length'].max()} символов")
        
        # Показываем первые 10 результатов
        print("\n🔍 Первые 10 результатов:")
        display(df_results.head(10))
        
    else:
        print("❌ Изображения не найдены в папке test_images")
else:
    print("❌ Модель или папка с изображениями не найдены")

## 💾 Сохранение результатов

In [ ]:
if 'df_results' in locals() and not df_results.empty:
    # Сохраняем результаты в CSV
    csv_path = os.path.join(OUTPUT_PATH, "predictions.csv")
    df_results.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"💾 Результаты сохранены в {csv_path}")
    
    # Сохраняем текстовый файл с результатами
    txt_path = os.path.join(OUTPUT_PATH, "predictions.txt")
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write("Результаты предсказаний CAPTCHA OCR\n")
        f.write("=" * 50 + "\n\n")
        for _, row in df_results.iterrows():
            f.write(f"{row['filename']}: {row['prediction']}\n")
    print(f"💾 Текстовый файл сохранен в {txt_path}")
    
    # Показываем примеры визуализации
    print("\n🖼️ Примеры визуализации:")
    sample_results = df_results.head(5)
    
    for _, row in sample_results.iterrows():
        if row['prediction']:  # Только для успешных предсказаний
            image_path = os.path.join(TEST_IMAGES_PATH, row['filename'])
            if os.path.exists(image_path):
                save_path = os.path.join(OUTPUT_PATH, f"result_{row['filename']}")
                visualize_prediction(image_path, row['prediction'], save_path)
    
    print(f"\n✅ Все результаты сохранены в папку: {OUTPUT_PATH}")
    print("   - predictions.csv - таблица с результатами")
    print("   - predictions.txt - текстовый файл с результатами")
    print("   - result_*.png - изображения с предсказаниями")
    
else:
    print("❌ Нет результатов для сохранения")

## 📋 Итоговая сводка

In [ ]:
print("🎉 Предсказания завершены!")
print("=" * 50)

if 'df_results' in locals() and not df_results.empty:
    total_images = len(df_results)
    successful = len(df_results[df_results['prediction'] != ''])
    success_rate = successful / total_images * 100
    
    print(f"📊 Обработано изображений: {total_images}")
    print(f"✅ Успешных предсказаний: {successful}")
    print(f"📈 Процент успеха: {success_rate:.1f}%")
    print(f"💾 Результаты сохранены в: {OUTPUT_PATH}")
    
    # Показываем примеры предсказаний
    print("\n🔍 Примеры предсказаний:")
    sample_predictions = df_results[df_results['prediction'] != ''].head(10)
    for _, row in sample_predictions.iterrows():
        print(f"   {row['filename']}: '{row['prediction']}'")
    
    if len(sample_predictions) < len(df_results[df_results['prediction'] != '']):
        print(f"   ... и еще {len(df_results[df_results['prediction'] != '']) - len(sample_predictions)} предсказаний")
    
else:
    print("❌ Предсказания не были выполнены")
    print("   Проверьте наличие модели и тестовых изображений")

print("\n🚀 Готово! Модель успешно протестирована на Kaggle.")